In [1]:
import os
import sys
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

import plotly
import plotly.graph_objs as go
import cufflinks as cf
cf.set_config_file(offline=True)

In [6]:
base_path = os.path.abspath('../../..')
data_path = os.path.join(base_path, 'data')
database_path = os.path.join(data_path, 'database')
strategy_path = os.path.join(base_path, 'strategy')
check_path = os.path.join(strategy_path, 'check')
sys.path.append(strategy_path)

In [7]:
from strategy import CommodityStrategy, EquityStrategy, IRStrategy, EmergingStrategy
from tester import Tester

In [18]:
# Bond data
JGB_flag = False

bond = pd.read_csv(os.path.join(data_path, 'bonds.csv'), header=0, index_col=0, parse_dates=True)
BRet = bond.pct_change(1)
Bindex = (1. + BRet).cumprod().iloc[1:]
Bindex = Bindex / Bindex.iloc[0]
BRet = BRet.iloc[1:]

In [19]:
CS=0.5
nopos=0.4
minobs=260
minobs1=52
longlen=52
shortlen=4

In [20]:
ret = BRet
index = Bindex

In [21]:
eindex_path = os.path.join(data_path, 'totindex.csv')
eindex1_path = os.path.join(data_path, 'priceindex.csv')
efuture_path = os.path.join(data_path, 'FutGenratio1.csv')

eindex = pd.read_csv(eindex_path, header=0, index_col=0, parse_dates=True)
eindex1 = pd.read_csv(eindex1_path, header=0, index_col=0, parse_dates=True)
efut = pd.read_csv(efuture_path, header=0, index_col=0, parse_dates=True)

eindex1 = eindex1.loc[:'2008']
ERetp = eindex1.pct_change(1).iloc[1:]
ERett = eindex.pct_change(1).iloc[1:]
fRet = efut.pct_change(1).iloc[1:]

compRet = ERett
compRet[compRet.isna()] = ERetp[compRet.isna()]
ERet1 = compRet
ERet1.drop_duplicates(inplace=True)
fRet.drop_duplicates(inplace=True)
ERet = pd.concat([ERet1.loc[:'2007-12-31'], fRet.loc['2008-01-01':]], axis=0)
ERet.fillna(0)
Eindex = (1 + ERet).cumprod()
Eindex = Eindex / Eindex.iloc[0]

In [22]:
if not JGB_flag:
    selected_columns = ['BOND', 'CANA',  'BUND', 'GILT']
    dm_idx_tickers = [u'SPX', u'TSX', u'DAX', u'FTSE']

    index = index[selected_columns]
    ret = ret[selected_columns]
else:
    dm_idx_tickers = [u'SPX', u'TSX', u'DAX', u'FTSE', u'NKY']

RET = ret

# Making Signal
Eqindex = Eindex.loc[Eindex.groupby(Eindex.index.to_period('M')).apply(
    lambda x: x.index.max()), dm_idx_tickers]  # to monthly, and selected ticker

In [23]:
Eqindex.head()

,SPX,TSX,DAX,FTSE
1988-01-29,1.004415,0.944231,0.981618,1.027740
1988-02-29,1.046417,0.992707,1.144167,1.016373
1988-03-31,1.011526,1.014708,1.121194,0.998704
1988-04-29,1.021060,1.022687,1.110975,1.026651
1988-05-31,1.024303,0.988924,1.130186,1.015067


,SPX,TSX,DAX,FTSE
1992-01-31,-1.054099,-1.025504,-1.016884,-0.983200
1992-02-28,-1.043674,-1.022663,-1.044490,-0.961825
1992-03-31,-1.040814,-1.007784,-1.054784,-0.921258
1992-04-30,-1.057305,-0.963201,-1.073060,-1.029838
1992-05-29,-1.106951,-0.990914,-1.121882,-1.123302
...,...,...,...,...
2018-01-31,-1.144581,-1.069715,-1.089811,-1.034831
2018-02-28,-1.098626,-1.033481,-1.031591,-0.984920
2018-03-30,-1.047787,-0.994206,-0.946222,-0.973131
2018-04-30,-1.026224,-0.980735,-0.953022,-1.018861


In [37]:
RV1 = (Eqindex.pct_change(1)) * - 1
RV1 = RV1.loc[index.index[0]:]

In [38]:
RV1.head()

,SPX,TSX,DAX,FTSE
1992-01-31,0.019900,-0.027069,-0.066688,-0.031595
1992-02-28,-0.009565,0.003292,-0.039326,-0.001432
1992-03-31,0.021832,0.050526,0.018519,0.047967
1992-04-30,-0.027893,0.010087,-0.000351,-0.094686
1992-05-29,-0.000964,-0.009570,-0.031073,-0.021390


In [31]:
RV = RV.loc[index.index[0]:]

In [32]:
RV

,SPX,TSX,DAX,FTSE
1992-01-31,0.075808,0.080752,0.135141,0.001542
1992-02-28,0.153127,0.063657,0.192028,0.018527
1992-03-31,-0.013146,-0.070777,0.124372,-0.143190
1992-04-30,0.100267,-0.110083,0.093849,0.168194
1992-05-29,0.114336,-0.050625,0.165290,0.209161
...,...,...,...,...
2018-01-31,0.297747,0.055674,0.108119,0.019973
2018-02-28,0.083314,-0.035514,-0.072406,-0.053924
2018-03-30,0.006107,-0.058376,-0.142380,-0.120442
2018-04-30,-0.036957,-0.024170,-0.050801,0.096817


In [ ]:

pctrank = lambda x: pd.Series(x).rank(pct=True, ascending=False).iloc[-1]
RVrank = RV.expanding(min_periods=12).apply(pctrank)  # it takes some time
RVrank.columns = RET.columns

In [ ]:


# Time Series
TSRV = RVrank * 0.
TSRV[RVrank > (nopos + (1. - nopos) / 2.)] = 1.
TSRV[RVrank < ((1. - nopos) / 2.)] = -1.
TSRV.dropna(how='all', inplace=True)

# Cross Sectional
truecount = (RVrank.notnull().sum(axis=1) * CS).apply(round)
tiebreaker = RVrank.rolling(5).mean() * 0.0000001
tiebreaker.iloc[:4] = 0
tied_RVrank = RVrank + tiebreaker

# 1. Cross sectional
CSRV = tied_RVrank.rank(axis=1, method='first')  # Short
CSRV1 = (CSRV.count(1).T + 1 - CSRV.T).T

CSRVpos = CSRV.fillna(0) * 0
CSRVpos[CSRV.apply(lambda x: x <= truecount, axis=0)] = -1
CSRVpos[CSRV1.apply(lambda x: x <= truecount, axis=0)] = 1
CSRV = CSRVpos
CSRV.fillna(0, inplace=True)

# Align dates with Return DataFrame
TSRV = TSRV.loc[ret.index].fillna(method='ffill').dropna(how='all')
CSRV = CSRV.loc[ret.index].fillna(method='ffill').dropna(how='all')

# Align dates with each other
if TSRV.index[0] > CSRV.index[0]:
    CSRV = CSRV.loc[TSRV.index[0]:]
else:
    TSRV = TSRV.loc[CSRV.index[0]:]